# Importación de librerías

In [225]:
import json
import pandas as pd
import numpy as np
from datetime import date, time, datetime

# Carga de datos

In [367]:
data1 = []
with open(r"D:\Users\user\Documents\PROYECTOS_DS\Anexos_Proyectos\EDA_geolocalizacion_Maps\Records.json") as data_file:    
    data = json.load(data_file)
    for location in data['locations']:
        data1.append(location)

In [368]:
df = pd.DataFrame(data=data1)
df.head()

,latitudeE7,longitudeE7,accuracy,activity,source,deviceTag,timestamp,deviceDesignation
0,-346437832,-584045990,29,"[{'activity': [{'type': 'IN_VEHICLE', 'confide...",WIFI,36907970,2016-05-04T04:16:45.310Z,NaN
1,-346437627,-584046859,24,NaN,WIFI,36907970,2016-05-04T04:17:30.697Z,NaN
2,-346437556,-584046358,24,"[{'activity': [{'type': 'IN_VEHICLE', 'confide...",WIFI,36907970,2016-05-04T04:19:04.274Z,NaN
3,-346436580,-584046147,43,"[{'activity': [{'type': 'TILTING', 'confidence...",WIFI,36907970,2016-05-04T04:19:44.676Z,NaN
4,-346437130,-584046736,42,"[{'activity': [{'type': 'STILL', 'confidence':...",WIFI,36907970,2016-05-04T04:21:29.709Z,NaN


# Data cleaning

## Atributos de las variables:

Sabemos con .shape que tenemos 515.570 datos en nuestro dataframe con 8 variables, además con .dtypes conocemos los nombres de cada variable  y el tipo de datos.

In [4]:
print(df.shape)
print(df.dtypes)

(515570, 8)
latitudeE7            int64
longitudeE7           int64
accuracy              int64
activity             object
source               object
deviceTag             int64
timestamp            object
deviceDesignation    object
dtype: object


Las variables "Activity" y "DeviceDesignation" tienen gran cantidad de valores nulos. Retomaremos esta parte en la siguiente sección.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515570 entries, 0 to 515569
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   latitudeE7         515570 non-null  int64 
 1   longitudeE7        515570 non-null  int64 
 2   accuracy           515570 non-null  int64 
 3   activity           231080 non-null  object
 4   source             515570 non-null  object
 5   deviceTag          515570 non-null  int64 
 6   timestamp          515570 non-null  object
 7   deviceDesignation  57121 non-null   object
dtypes: int64(4), object(4)
memory usage: 31.5+ MB


Entonces, las variables son: 

1. LatitudeE7: Latitud con formato E7.

2. LongitudeE7: Longitud con formato E7.

3. Accuracy: Es el nivel de precisión de la geolocalización.

4. Activity:

5. Source: Indica la fuente que se utilizó para generar el registro de geolocalización.

6. DeviceTag: Identificador entero (específico del historial de ubicaciones) asociado con el dispositivo que cargó la ubicación. 

7. Timestamp: Da como dato la fecha y hora exacta en que empieza la acción que activa la aplicación en hora UTC+0.

8. DeviceDesignation: campo con descripción desconocida.

## Limpieza

### 1. LatitudeE7
### 2. LongitudeE7:

EL formato de Longitud y Latitud es de E7, quiere decir que está eliminando los valores decimales. Para volver a tenerlos, debemos multiplicarlo por 0.0000001.

In [6]:
df['latitudeE7'] = df['latitudeE7']*0.0000001
df['longitudeE7'] = df['longitudeE7']*0.0000001
df[['latitudeE7', 'longitudeE7']].head()

,latitudeE7,longitudeE7
0,-34.643783,-58.404599
1,-34.643763,-58.404686
2,-34.643756,-58.404636
3,-34.643658,-58.404615
4,-34.643713,-58.404674


### 3. Accuracy:

In [7]:
mx_acur = max(df['accuracy'])
mn_acur = min(df['accuracy'])
print(f'el máximo valor es {mx_acur} y el mínimo es {mn_acur}')

el máximo valor es 58619 y el mínimo es 0


Vemos que el nivel de presición no tiene una valoración estándar de calificación. Haremos un análisis de la concentración de datos en la sección de gráficos y estadística descriptiva.

### 4. Activity:

In [372]:
df['activity'].apply(lambda x: [np.nan] if pd.isnull(x) else x))

SyntaxError: invalid syntax (<ipython-input-372-904ac2e09e99>, line 1)

In [369]:
df['activity'].fillna([0,0], inplace=True)
df.head()

TypeError: "value" parameter must be a scalar or dict, but you passed a "list"

In [338]:
Actividad = list(df['activity'])
#Actividad[0][0]['activity'][0]['type']
#Actividad[17][2]['activity'][1]['type']
Actividad[17][0]['activity'][0]['confidence']
Actividad[17][1]['activity'][0]['confidence']
Actividad[17][2]['activity'][0]['confidence']

85

In [339]:
len(Actividad)

515570

In [ ]:
for n in Actividad[17]:
    Actividad[17][0]['activity'][0]['confidence']
    Actividad[17][n]['activity'][0]['confidence']


In [356]:
len(Actividad[1])

TypeError: object of type 'int' has no len()

In [355]:
def mayior(y):
    n = 0
    lista_confianza = []
    lista_tipo = []
    cant_elem = len(Actividad[y])

    while n < cant_elem:
        lista_confianza.append(Actividad[y][n]['activity'][0]['confidence'])
        lista_tipo.append(Actividad[y][n]['activity'][0]['type'])
        n += 1
        
    maximo = max(lista_confianza)
    lista_integrada = [lista_tipo[lista_confianza.index(maximo)], maximo]
    return(lista_integrada)

mayior(1)

TypeError: object of type 'int' has no len()

In [320]:
def mayior(y):
    elemento = Actividad[y]
    n = 0
    lista_confianza = []
    lista_tipo = []
    cant_elem = len(Actividad[y])

    if np.isnan(elemento) == False:
        while n < cant_elem:
            lista_confianza.append(Actividad[y][n]['activity'][0]['confidence'])
            lista_tipo.append(Actividad[y][n]['activity'][0]['type'])
            n += 1
        
        maximo = max(lista_confianza)
        lista_integrada = [lista_tipo[lista_confianza.index(maximo)], maximo]
        return(lista_integrada)
    else:
        return([0,0])

mayior(17)
    
    

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [297]:
Actividad[1]

nan

In [314]:
a = 10
b = 0
if np.isnan(Actividad[1]) == False:
    print("Ya lo sabia")
else:
    print("pucha hoy no sale")

pucha hoy no sale


### 5. Source:

In [9]:
pd.value_counts(df.source)

WIFI       365692
GPS        109055
CELL        40818
UNKNOWN         5
Name: source, dtype: int64

### 6. DeviceTamp:

Se sabe que es un identificador de dispositivo, pero desconocemos a ciencia cierta el significado de su valor entero y su signo. Sin embargo, podemos saber que se trata únicamente de 9 valores que se van repitiendo en los registros.

In [10]:
df['deviceTag'].nunique()

9

In [11]:
pd.value_counts(df['deviceTag'])

-1615259547    233018
 299628238     138139
 1814650302     69870
-1013692988     38856
 36907970       17808
-523263708      10903
 1040228492      5842
-263022313       1124
 1749122152        10
Name: deviceTag, dtype: int64

### 7. Timestamp:

Está en formato "object" y debemos cambiarlo a formato "YYYY-mm-ddTHH:mm:ssZ", en el cual T sirve como separador de la fecha y la hora y Z indica que está en la hora UTC.

In [12]:
df['timestamp']= pd.to_datetime(df['timestamp'], format=('%Y-%m-%dT%H:%M:%S.%f'))
df['timestamp']

0        2016-05-04 04:16:45.310000+00:00
1        2016-05-04 04:17:30.697000+00:00
2        2016-05-04 04:19:04.274000+00:00
3        2016-05-04 04:19:44.676000+00:00
4        2016-05-04 04:21:29.709000+00:00
                       ...               
515565          2020-07-31 15:38:12+00:00
515566          2020-07-31 15:38:28+00:00
515567          2020-07-31 15:38:32+00:00
515568          2020-07-31 15:38:41+00:00
515569          2020-07-31 15:38:42+00:00
Name: timestamp, Length: 515570, dtype: datetime64[ns, UTC]

En nuestro análisis podríamos usar los datos de año, mes, día y hora. No consideramos necesario los minutos y segundos. Además nos percatamos que originalmente el formato incluía Z, es decir estaba en hora UTC, debemos incluir el cambio horario según nuestra ubicación geográfica. Sabemos que durante el lapso del tiempo de análisis, no hubo salidas al exterior por lo que siempre se mantuvo en UTC-3 que pertenece a Argentina, cambiamos la hora y le restamos 3 para conocer la hora local.

In [13]:
df['year'] = pd.DatetimeIndex(df['timestamp']).year
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df['day'] = pd.DatetimeIndex(df['timestamp']).day
df['hour'] = pd.DatetimeIndex(df['timestamp']).hour - 3

df[['year', 'month', 'day', 'hour']]


,year,month,day,hour
0,2016,5,4,1
1,2016,5,4,1
2,2016,5,4,1
3,2016,5,4,1
4,2016,5,4,1
...,...,...,...,...
515565,2020,7,31,12
515566,2020,7,31,12
515567,2020,7,31,12
515568,2020,7,31,12


### 8. DeviceDesignation:

Desconocemos su utilidad, pero podemos saber que los valores únicos obtenidos en este campo son PRIMARY y UNKNOWN. Debido a la cantidad de datos nulos y el desconocimiento de su uso, se toma la decisición de eliminar esta variable dentro del análisis.

In [14]:
pd.value_counts(df['deviceDesignation'])

PRIMARY    56783
UNKNOWN      338
Name: deviceDesignation, dtype: int64

Porbando si actualiza